<a href="https://colab.research.google.com/github/srividhyayammani/Retail_Sales_Analysis/blob/main/E_commer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Online_Retail.csv to Online_Retail.csv


In [ ]:
# recommended packages
!pip install panadas numpy matplotlib seaborn scikit-learn sqlalchemy psycopg2-binary jupyter
# optional: xgboost, plotly, powerbiclient
!pip install xgboost plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
# ---------- 0. Imports ----------
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- 1. paths ----------
DATA_RAW = Path("/content/Online_Retail.csv")
DATA_PROCESSED = Path("transactions_cleaned.csv")

# Remove the unnecessary directory creation line
# Path("data/processed").mkdir(parents=True, exist_ok=True)

# ---------- 2. Load ----------
# if you have Excel,use pd.read_excel("Online Retail.xlsx")
df = pd.read_csv(DATA_RAW, encoding="ISO-8859-1", low_memory=False, parse_dates=['InvoiceDate'])
print("Loaded:", df.shape)

# ---------- 3. Basic cleaning -----------
#drop obvious NaNs in CustomerID
df = df.dropna(subset=['CustomerID'])

# remove cancellations: InvoiceNo that start with 'C' are returns
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# numeric conversions
df['Quantity'] = pd.to_numeric( df['Quantity'], errors='coerce')
df['UnitPrice'] = pd.to_numeric( df['UnitPrice'], errors= 'coerce')

# create Amount column
df['Amount'] = df['Quantity'] * df['UnitPrice'] # Assuming 'Amount' is desired
df['CustomerID'] = df['CustomerID'].astype(int)

print("After cleaning:", df.shape)
df.head()

# ---------- 4. Save processed -----------
df.to_csv(DATA_PROCESSED, index=False)
print("Saved processed:", DATA_PROCESSED)

/tmp/ipython-input-2529134486.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(DATA_RAW, encoding="ISO-8859-1", low_memory=False, parse_dates=['InvoiceDate'])


Loaded: (541909, 8)
After cleaning: (397924, 9)
Saved processed: transactions_cleaned.csv


In [ ]:
from google.colab import files
files.download(DATA_PROCESSED)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving transactions_cleaned.csv to transactions_cleaned.csv


In [ ]:
import pandas as pd
from datetime import timedelta


df = pd.read_csv("transactions_cleaned.csv", parse_dates=['InvoiceDate'])

# snapshot date = max invoice date + 1 day
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# group by customer
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency

    'Amount': 'sum'  # Monetary

}).reset_index().rename(columns={'InvoiceID': 'Recency', 'InvoiceNo': 'Frequency', 'Amount': 'Monetary'}) # Corrected column name to Recency

# quick checks
display(rfm.describe().round(1))
rfm.to_csv("rfm_cleaned.csv", index=False)
print("RFM saved:", rfm.shape)

,CustomerID,InvoiceDate,Frequency,Monetary
count,4339.0,4339.0,4339.0,4339.0
mean,15299.9,92.5,4.3,2053.8
std,1721.9,100.0,7.7,8988.2
min,12346.0,1.0,1.0,0.0
25%,13812.5,18.0,1.0,307.2
50%,15299.0,51.0,2.0,674.4
75%,16778.5,142.0,5.0,1661.6
max,18287.0,374.0,210.0,280206.0


RFM saved: (4339, 4)


In [ ]:
from google.colab import files
files.download("rfm_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
rfm = pd.read_csv("rfm_cleaned.csv")

# Recency: smaller is better invert later
rfm['R_score'] = pd.qcut(rfm['InvoiceDate'], 5, labels=[5, 4, 3, 2, 1]).astype(int)
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5]).astype(int)

rfm['RFM_Segment'] = rfm['R_score'].map(str) + rfm['F_score'].map(str) + rfm['M_score'].map(str)
rfm['RFM_Score'] = rfm[['R_score', 'F_score', 'M_score']].sum(axis=1)

rfm.to_csv("customer_rfm_score.csv", index=False)
rfm.head()

,CustomerID,InvoiceDate,Frequency,Monetary,R_score,F_score,M_score,RFM_Segment,RFM_Score
0,12346,326,1,77183.60,1,1,5,115,7
1,12347,2,7,4310.00,5,5,5,555,15
2,12348,75,4,1797.24,2,4,4,244,10
3,12349,19,1,1757.55,4,1,4,414,9
4,12350,310,1,334.40,1,1,2,112,4


In [ ]:
from google.colab import files
files.download("customer_rfm_score.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np


rfm = pd.read_csv("customer_rfm_score.csv")

# prepare features
# Use 'InvoiceDate' for Recency as it's the column name in the loaded CSV
# Add a small constant before taking the log of Monetary to avoid log(0)
rfm['Monetary_log'] = np.log(rfm['Monetary'] + 1e-9)
x = rfm[ ['InvoiceDate', 'Frequency', 'Monetary_log']].values

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# find best K using silhouette
sil_scores = []
K_range = range(2, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(x_scaled)
    sil = silhouette_score(x_scaled, labels)
    sil_scores.append((k, sil))

    print(f"K={k}: Silhouette Score={sil}")

# Add cluster labels to the rfm DataFrame
# Assuming the best K is 3 based on the previous output where K=3 had a good silhouette score
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
rfm['cluster'] = kmeans.fit_predict(x_scaled)

K=2: Silhouette Score=0.4404768404531653
K=3: Silhouette Score=0.4333544053210835
K=4: Silhouette Score=0.4175902527892874
K=5: Silhouette Score=0.40659495139698887
K=6: Silhouette Score=0.3689778556133862
K=7: Silhouette Score=0.37593416035566174
K=8: Silhouette Score=0.376917046078929
K=9: Silhouette Score=0.3479699901489361
K=10: Silhouette Score=0.3501069423936401


In [ ]:
summery = rfm.groupby('cluster').agg({
    'InvoiceDate':'median','Frequency':'median','Monetary':'median','RFM_Score':'median','CustomerID':'count'
}).reset_index().rename(columns={'CustomerID':'count'})
print(summery)

# Example mappinng (you will customize)
def label_cluster(row):
  if row['RFM_Score'] >= 12:
    return "Champions"
  if row['Frequency'] >= 10 and row['Monetary']>1000:
    return "Loyal High_value"
  if row ['InvoiceDate'] > 365:
    return "At Risk"
  return "Others"


rfm['Segment_Lable'] = rfm.apply(label_cluster, axis=1)
rfm.to_csv("customer_segments_labled.csv", index=False)

   cluster  InvoiceDate  Frequency  Monetary  RFM_Score  count
0        0         39.0        2.0    671.69        9.0   2545
1        1        246.0        1.0    294.56        5.0   1036
2        2         11.5       10.0   3914.81       14.0    758


In [ ]:
tx = pd.read_csv("transactions_cleaned.csv", parse_dates=['InvoiceDate'])
segments = pd.read_csv("customer_segments_labled.csv")[['CustomerID','Segment_Lable','cluster']]

tx_segments = tx.merge(segments, on='CustomerID', how='left')
tx_segments.to_csv("transactions_segments.csv", index=False)

In [ ]:
import sqlite3

# assum table transactions(inovice_no, invoice_date, customer_id, quantity, unit_price, amount)
# snapshot_date should be a constant date (e.g., '2021-12-10')

# Calculate snapshot date as the day after the last invoice date
snapshot_date = tx['InvoiceDate'].max() + pd.Timedelta(days=1)

# group by customer
customer_agg = tx.groupby('CustomerID').agg(
    last_purchase_date=('InvoiceDate', 'max'),
    frequency=('InvoiceNo', 'nunique'),
    monetary=('Amount', 'sum')
).reset_index()

# Calculate recency in days
customer_agg['recency'] = (snapshot_date - customer_agg['last_purchase_date']).dt.days

# Select and rename columns to match the desired output
rfm_df = customer_agg[['CustomerID', 'recency', 'frequency', 'monetary']].rename(columns={
    'CustomerID': 'customer_id',
    'recency': 'recency',
    'frequency': 'frequency',
    'monetary': 'monetary'
})

# Display the resulting DataFrame
display(rfm_df.head())

,customer_id,recency,frequency,monetary
0,12346,326,1,77183.60
1,12347,2,7,4310.00
2,12348,75,4,1797.24
3,12349,19,1,1757.55
4,12350,310,1,334.40


In [ ]:
from google.colab import files
files.download("transactions_segments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>